## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 8
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_8


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 21.3 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Buffalo Bills,7,2022,29.0,60.0,8.1,307.7,104.0,12.6,14.0,...,10.3,95.0,19.857143,11.285714,12.000000,0.103704,2.714286,0.571429,1.750000,3.166667
1,Kansas City Chiefs,7,2022,31.9,60.0,8.1,296.0,108.3,12.3,24.6,...,10.5,92.0,20.857143,11.285714,12.857143,0.064935,2.857143,0.428571,5.666667,4.000000


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[7, 8]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_8,0.882223,0.370197,0.208095,0.317569,0.404456,0.436508,0.464177
1,Kansas City Chiefs,2022,week_8,0.485175,0.333554,0.256478,0.375437,0.418278,0.373784,0.426640
2,Philadelphia Eagles,2022,week_8,0.464398,0.330753,0.168044,0.337446,0.362378,0.332604,0.375131


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Buffalo Bills,2022,week_8,0.882223,0.370197,0.208095,0.317569,0.404456,0.436508,0.464177
1,Kansas City Chiefs,2022,week_8,0.485175,0.333554,0.256478,0.375437,0.418278,0.373784,0.426640
2,Philadelphia Eagles,2022,week_8,0.464398,0.330753,0.168044,0.337446,0.362378,0.332604,0.375131
23,Dallas Cowboys,2022,week_8,0.241246,0.153300,0.090514,0.207929,0.355436,0.209685,0.341162
10,San Francisco 49ers,2022,week_8,0.213895,0.084281,0.080577,0.209658,0.350856,0.187854,0.333736
20,Tampa Bay Buccaneers,2022,week_8,0.156434,0.069978,0.051842,0.180514,0.349502,0.161654,0.325369
22,Denver Broncos,2022,week_8,0.083660,0.153300,0.097488,0.199308,0.346480,0.176047,0.313627
6,Miami Dolphins,2022,week_8,0.028244,0.218326,0.084763,0.234130,0.348221,0.182737,0.308224
9,Baltimore Ravens,2022,week_8,0.013479,0.039408,0.044244,0.180358,0.347685,0.125035,0.305909
12,Cincinnati Bengals,2022,week_8,0.014460,0.096144,0.060870,0.204889,0.343700,0.144012,0.302545
